In [1]:
import parse_tfl_json
import google

In [26]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime_cw where tfl_message = 'ok'
"""
df = pd.read_sql(sql, conn)

In [29]:
import json 

for r in df.iterrows():
    index = r[0]
    row = r[1]

    j1 = row["tfl_response"]
    j2 = json.loads(j1)
    
    try:
        journeys = j2["journeys"]
        journey = sorted(journeys, key = lambda x:  x["duration"])[0]

        legs = journey["legs"]

        legs = parse_tfl_json.remove_non_national_rail(legs)

        summary = parse_tfl_json.summarise_legs(legs)
        df.loc[index, "natrail_journey_summary"] = summary

        df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
        df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

        travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
        df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

        lat_lng = parse_tfl_json.lat_lng_from_legs(legs)
        df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
        df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
        df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
        df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]
    except Exception as e:
        df.loc[index, "error"] = "Parse problem"
    
    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict", "journey_time", "london_mainline_station_ics", "london_mainline_station_text"]]
df2 = df[cols]

In [51]:
df3 = df2.sort_values(["station_name", "natrail_journey_minutes"]).drop_duplicates("station_name")

In [61]:
df_arrivals = df2.drop_duplicates("final_arrival")[["final_arrival", "arrive_lat", "arrive_lng"]]


In [64]:
# Next - need a bikedistance function that takes a lat lng and looks up 

for r in df_arrivals.iterrows():
    index = r[0]
    row = r[1]
    
    lat = row["arrive_lat"]
    lng = row["arrive_lng"]
    
    try:
        cycle_info = get_cycle_info(origin_lat = lat, origin_lng = lng, dest_pc= "E145HP")
    
        df_arrivals.loc[index, "cycle_miles_london"] = cycle_info["miles"]
        df_arrivals.loc[index, "cycle_minutes_london"] = cycle_info["minutes"]
    except Exception as e:
        pass


{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['22 Great Eastern Rd, London E15 1BB, UK'], 'rows': [{'elements': [{'distance': {'text': '3.9 mi', 'value': 6271}, 'duration': {'text': '16 mins', 'value': 976}, 'status': 'OK'}]}], 'status': 'OK'}
{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['1 Leake St, Lambeth, London SE1 7NN, UK'], 'rows': [{'elements': [{'distance': {'text': '6.2 mi', 'value': 9925}, 'duration': {'text': '34 mins', 'value': 2060}, 'status': 'OK'}]}], 'status': 'OK'}
{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['20 York Way, Kings Cross, London N1 9AA, UK'], 'rows': [{'elements': [{'distance': {'text': '5.8 mi', 'value': 9352}, 'duration': {'text': '37 mins', 'value': 2229}, 'status': 'OK'}]}], 'status': 'OK'}
{'destination_addresses': ['London E14 5HP, UK'], 'origin_addresses': ['11 London St, London W2, UK'], 'rows': [{'elements': [{'distance': {'text': '8.3 mi', 'value': 13365}, 'duration': 

In [97]:
df_arrivals = df_arrivals[["final_arrival", "cycle_miles_london", "cycle_minutes_london"]]
df4 = df3.merge(df_arrivals, how = "left", left_on = "final_arrival", right_on = "final_arrival")

In [100]:
keep = ["station_name", "natrail_journey_summary", "depart_lat", "depart_lng", "natrail_journey_minutes","cycle_minutes_london"]
df4 = df4[keep]
df4["total_minutes"] = df4["cycle_minutes_london"] + df4["natrail_journey_minutes"]
df4.to_csv("cw_to_map.csv", encoding="ut")

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,station_name,natrail_journey_summary,depart_lat,depart_lng,natrail_journey_minutes,cycle_minutes_london,total_minutes
0,Abbey Wood,Abbey Wood → Woolwich Arsenal Station,51.491111,0.121123,4.0,25.964431,29.964431
1,Aber,Aber Rail Station → Cardiff Central Rail Stati...,51.574964,-3.229828,187.0,35.591243,222.591243
2,Abercynon,Abercynon North Rail Station → Abercynon Rail ...,51.645312,-3.326803,220.0,35.591243,255.591243
3,Aberdare,Aberdare Rail Station → Cardiff Central Rail S...,51.715056,-3.443083,248.0,35.591243,283.591243
4,Aberdeen,Aberdeen Rail Station → London King's Cross Ra...,57.143703,-2.098685,419.0,24.904550,443.904550
5,Aberdour,Aberdour Rail Station → Edinburgh Rail Station...,56.054590,-3.300560,307.0,24.904550,331.904550
6,Aberdovey,Aberdovey Rail Station → Wolverhampton Rail St...,52.543955,-4.057067,263.0,27.796588,290.796588
7,Abererch,Abererch Rail Station → Shrewsbury Rail Statio...,52.898581,-4.374175,376.0,27.796588,403.796588
8,Abergavenny,Abergavenny Rail Station → Newport (S Wales) R...,51.816688,-3.009647,177.0,35.591243,212.591243
9,Abergele & Pensarn,Abergele & Pensarn Rail Station → Chester Rail...,53.294571,-3.582622,175.0,27.796588,202.796588
